# Multi-Objective Exponential Function
# optimized using MOO Easy Tool

## Initialization

### MOO imports

In [ ]:
import requests
import os
## Set API back-end direction
backend="http://138.100.20.80/api/"

### Create new experiment

In [ ]:
experiment = {
    "name": "MOExponential_2inputd",
    "n_ins": 1, 
    "input_names": ["x1", "x2"], 
    "input_mms": [[-2, 2],[-2, 2]],
    "n_objs": 2,
    "objective_names": ["f1(x)","f2(x)"],
    "objective_mms": [False, False],
    "n_cons": 0,
    "kernel": 'RBF',
    "acq_id": 2,
    "acqfunct_hps": {"N": 1000, "M": 50}
}

response = requests.post(backend+'startexp', json=experiment)
print(response.__dict__)
if response.status_code==200:
    response = response.json()
    for key in response:
        experiment[key]=response[key]

print(experiment)

### Define Evaluation Function

In [ ]:
import numpy as np

def MOOexponential(xx, c1=-0.5, c2=0.5):
    return np.array([1-np.exp(-np.sum((xx-c1)**2)), 1-np.exp(-np.sum((xx-c2)**2))])

def evaluation(x):
    return MOOexponential(x, c1=-.6, c2=.6)

## First iteration

In [ ]:
x_best = np.random.uniform(experiment["input_mms"][0][0], experiment["input_mms"][0][1], experiment["n_ins"])
y_best = evaluation(x_best)
response = requests.post(backend+"setsample/"+str(experiment["id"]), json={"x": list(x_best), "y": list(y_best)})

## Main Loop

In [ ]:
for i in range(49):
    response = requests.get(backend+"getsample/next/"+str(experiment["id"]))

    ## Charge next configuration 
    x_best = np.array(response.json()["next_x"])
    y_best = evaluation(x_best)

    response = requests.post(backend+"setsample/"+str(experiment["id"]), json={"x": list(x_best), "y": list(y_best)})